In [ ]:
import numpy as np
from section4_7_common import get_Q_out_H_hs_d_t

### 付録E　電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機（給湯熱源：ガス瞬間式、暖房熱源：電気ヒートポンプ・ガス瞬間式併用）

#### E.1 記号及び単位

省略

#### E.2 エネルギー消費量

#### E.2.1 消費電力量

1 時間当たりの熱源機の消費電力量$E_{E,hs}$は、1 時間当たりの熱源機の暖房出力$Q_{out,hs}$が0の場合は0とする。

$$\large
E_{E,hs} = E_{E,HPU} + E_{E,GU,aux} + E_{E,TU,aux}
$$

$E_{E,hs}$：1 時間当たりの熱源機の消費電力量(kWh/h)  
$E_{E,HPU}$：1 時間当たりのヒートポンプユニットの消費電力量(kWh/h)  
$E_{E,GU,aux}$：1 時間当たりのガスユニットの補機の消費電力量(kWh/h)  
$E_{E,TU,aux}$：1 時間当たりのタンクユニットの補機の消費電力量(kWh/h)  

In [ ]:
# 1時間当たりの熱源機の消費電力量 (1)
def get_E_E_hs(Q_dmd_H_hs_d_t, Theta_ex, Theta_RW_hs, h_ex, Theta_SW_d_t, TU_place):
    """
    :param Q_dmd_H_hs_d_t: 温水暖房用熱源機の温水熱需要 (MJ/h)
    :param Theta_ex: 1時間平均の外気温度 (℃)
    :param Theta_RW_hs: 1時間平均の熱源機の戻り温水温度 (℃)
    :param h_ex: 1時間平均の外気相対湿度 (%)
    :param Theta_SW_hs: 1時間平均の熱源機の往き温水温度 (℃)
    :param TU_place: タンクユニットの設置場所
    :return: 1時間当たりの熱源機の消費電力量 (kWh/h)
    """
    # ---------- タンクユニット周囲の空気温度 ----------

    # 1時間平均のタンクユニットの周囲空気温度 (℃) (23)
    Theta_TU_amb = get_Theta_TU_amb(Theta_ex, TU_place)

    # ---------- 熱源機の平均暖房出力及び熱源機最大暖房出力 ----------

    # ガスユニットの定格暖房能力 (W)
    q_GU_rtd = get_q_GU_rtd()

    # 1時間当たりの熱源機の最大暖房出力 (MJ/h) (22)
    Q_max_hs = get_Q_max_hs(q_GU_rtd)

    # 1時間当たりの温水暖房用熱源機の暖房出力 (MJ/h)
    Q_out_hs = get_Q_out_H_hs_d_t(Q_dmd_H_hs_d_t, Q_max_hs)

    # 1時間平均の熱源機の暖房出力 (W) (21)
    q_out_hs = get_q_out_hs(Q_out_hs)

    # ----------- 暖房出力分担率 -----------

    # ヒートポンプ最大運転時のヒートポンプユニット運転時のガスユニットの熱損失 (W) (20)
    q_dash_GU_loss_HPU = get_q_dash_GU_loss_HPU(Theta_SW_d_t, Theta_TU_amb)

    # ヒートポンプ最大運転時のタンクユニットのタンクの熱損失 (W) (19)
    q_dash_TU_tank_loss = get_q_dash_TU_tank_loss(Theta_TU_amb)

    # タンクユニットのヒートポンプ配管の熱損失 (W) (18)
    q_dash_TU_pipe_loss = get_q_dash_TU_pipe_loss(Theta_SW_d_t, Theta_ex)

    # ヒートポンプユニットの最大暖房出力 (W) (17)
    q_HPU_max = get_q_HPU_max(Theta_ex, Theta_RW_hs)

    # ヒートポンプユニットの暖房出力分担率 (-) (16)
    r_HPU = get_r_HPU(q_HPU_max, q_dash_TU_tank_loss, q_dash_TU_pipe_loss, q_out_hs, q_dash_GU_loss_HPU)

    # ガスユニットの暖房出力分担率 (-) (15)
    r_GU = get_r_GU(r_HPU)

    # ---------- ガスユニットのガス消費量 ----------

    # 1時間平均のガスユニットのガス消費量 (W) (14b)
    q_G_GU = get_q_G_GU(q_out_hs, Theta_SW_d_t)

    # ---------- ガスユニットの補機の消費電力 ----------

    # 1時間当たりのガスユニットの補機の消費電力 (kWh/h) (13)
    E_E_GU_aux = get_E_E_GU_aux(q_G_GU, r_HPU, r_GU)

    # ---------- ヒートポンプユニットの暖房出力 ----------

    # 熱源機のヒートポンプユニット分断暖房出力 (MJ/h) (12)
    Q_out_hs_HPU = get_Q_out_hs_HPU(Q_out_hs, r_HPU)

    # ヒートポンプユニット運転時のガスユニットの熱損失 (MJ/h) (11)
    Q_GU_loss_HPU = get_Q_GU_loss_HPU(Theta_SW_d_t, Theta_TU_amb, r_HPU)

    # 1時間平均のヒートポンプユニットの出湯温度 (℃)
    Theta_HPU_out = get_Theta_HPU_out(r_HPU, Theta_SW_d_t)

    # タンクユニットのタンクの熱損失 (MJ/h) (10)
    Q_TU_tank_loss = get_Q_TU_tank_loss(Theta_HPU_out, Theta_TU_amb)

    # タンクユニットのヒートポンプ配管の熱損失 (MJ/h) (9)
    Q_TU_pipe_loss = get_Q_TU_pipe_loss(Theta_SW_d_t, Theta_ex)

    # ヒートポンプユニットの暖房出力 (MJ/h) (8)
    Q_HPU = get_Q_HPU(Q_out_hs_HPU, Q_GU_loss_HPU, Q_TU_tank_loss, Q_TU_pipe_loss)

    # ---------- ヒートポンプユニットの消費電力量 ----------

    # ヒートポンプユニットの平均暖房出力,最小暖房出力 (6)(7)
    q_HPU_ave = get_q_HPU_ave(Q_HPU)
    q_HPU_min = get_q_HPU_min(Theta_ex, Theta_RW_hs, Theta_HPU_out)

    # 1時間平均のヒートポンプユニットの断続運転率 (-) (5)
    r_intmit = get_r_intmit(q_HPU_ave, q_HPU_min)

    # 1時間当たりのヒートポンプユニットの消費電力量 (kWh/h) (4)
    E_E_HPU = get_E_E_HPU(q_HPU_ave, q_HPU_min, Theta_ex, h_ex, Theta_RW_hs, Theta_HPU_out)

    # ---------- タンクユニットの補機消費電力量 ----------

    # 1時間当たりのタンクユニットの補機の消費電力量 (kWh/h) (3)
    E_E_TU_aux = get_E_E_TU_aux(r_intmit)

    # ---------- 熱源機の消費電力量 ----------

    # 1時間当たりの熱源機の消費電力量 (kWh/h) (1)
    E_E_hs = E_E_HPU + E_E_GU_aux + E_E_TU_aux
    E_E_hs[Q_out_hs == 0] = 0

    return E_E_hs

#### E.2.2 ガス消費量

1 時間当たりの熱源機のガス消費量$E_{G,hs}$は、1 時間当たりの熱源機の暖房出力$Q_{out,hs}$が0の場合は0とする。

$$\large
E_{G,hs} = E_{G,GU}
$$

$E_{G,hs}$：1 時間当たりの熱源機のガス消費量(MJ/h)  
$E_{G,GU}$：1 時間当たりのガスユニットのガス消費量(MJ/h)  

In [ ]:
# 熱源機のガス消費量 (2)
def get_E_G_hs(Theta_ex, Theta_SW_d_t, Theta_RW_hs, TU_place, Q_dmd_H_hs_d_t):
    # タンクユニットの周囲空気温度 (℃) (23)
    Theta_TU_amb = get_Theta_TU_amb(Theta_ex, TU_place)

    # ガスユニットの定格暖房能力 (W)
    q_GU_rtd = get_q_GU_rtd()

    # 熱源機の最大暖房出力 (22)
    Q_max_hs = get_Q_max_hs(q_GU_rtd)

    # 温水暖房用熱源機の暖房出力
    Q_out_hs = get_Q_out_H_hs_d_t(Q_dmd_H_hs_d_t, Q_max_hs)

    # 熱源機の暖房出力 (21)
    q_out_hs = get_q_out_hs(Q_out_hs)

    # ヒートポンプ最大運転時のヒートポンプユニット運転時のガスユニットの熱損失 (20)
    q_dash_GU_loss_HPU = get_q_dash_GU_loss_HPU(Theta_SW_d_t, Theta_TU_amb)

    # ヒートポンプ最大運転時のタンクユニットのタンクの熱損失 (19)
    q_dash_TU_tank_loss = get_q_dash_TU_tank_loss(Theta_TU_amb)

    # タンクユニットのヒートポンプ配管の熱損失 (18)
    q_dash_TU_pipe_loss = get_q_dash_TU_pipe_loss(Theta_SW_d_t, Theta_ex)

    # ヒートポンプユニットの最大暖房出力 (17)
    q_HPU_max = get_q_HPU_max(Theta_ex, Theta_RW_hs)

    # ヒートポンプユニットの暖房出力分担率 (16)
    r_HPU = get_r_HPU(q_HPU_max, q_dash_TU_tank_loss, q_dash_TU_pipe_loss, q_out_hs, q_dash_GU_loss_HPU)

    # ガスユニットの暖房出力分担率 (15)
    r_GU = get_r_GU(r_HPU)

    # ガスユニットのガス消費量 (14a)
    E_G_hs = get_E_G_GU(r_GU, q_out_hs, Theta_SW_d_t)
    E_G_hs[Q_out_hs == 0] = 0

    return E_G_hs

#### E.2.3 灯油消費量

1 時間当たりの熱源機の灯油消費量$E_{K,hs}$は0とする。 

In [ ]:
# 熱源機の灯油消費量
def get_E_K_hs():
    return np.zeros(24 * 365)

#### E.2.4 その他の燃料の一次エネルギー消費量

1 時間当たりの熱源機のその他の燃料の一次エネルギー消費量$E_{M,hs}$は0とする。 

In [ ]:
# 熱源機のその他の燃料の一次エネルギー消費量
def get_E_M_hs():
    return np.zeros(24 * 365)

#### E.3 消費電力量及びガス消費量

#### E.3.1 タンクユニットの補機消費電力量

$$\large
E_{E,TU,aux} = P_{TU,aux} \times 10^{-3}
$$

$$\large
P_{TU,aux} = f_{TU,P,aux} (r_{intmit})
$$

$P_{TU,aux}$：1 時間平均のタンクユニットの補機の消費電力(W)  
$r_{intmit}$：1 時間平均のヒートポンプユニットの断続運転率

In [ ]:
# 1時間当たりのタンクユニットの補機消費電力量 (3)
def get_E_E_TU_aux(r_intmit):
    """
    :param r_intmit: 1時間平均のヒートポンプユニットの断続運転率
    :return: 1時間当たりのタンクユニットの補機消費電力量 (kW)
    """
    # 1時間平均のタンクユニットの補機の消費電力 (3b)
    P_TU_aux = f_TU_P_aux(r_intmit)

    # タンクユニットの補機消費電力量 (3a)
    E_E_TU_aux = P_TU_aux * 10 ** (-3)

    return E_E_TU_aux

#### E.3.2 ヒートポンプユニットの消費電力量

$q_{HPU,ave} \geq q_{HPU,min} $ の場合（連続運転の場合）

$$\large
E_{E,HPU} = P_{HPU,ctn} \times 10^{-3}
$$

$$\large
P_{HPU,ctn} = f_{HPU,P} (\theta_{ex}, h_{ex}, \theta_{RW,hs}, \theta_{HPU,out}, q_{HPU,ave})
$$

$q_{HPU,ave} < q_{HPU,min} $ の場合（断続運転の場合）

$$\large
E_{E,HPU} = \frac{q_{HPU,ave}}{\frac{q_{HPU,min}}{P_{HPU,min}} \times (1 - CD \times (1 - r_{intmit}))} \times 10^{-3}
$$

$$\large
P_{HPU,min} = f_{HPU,P} (\theta_{ex}, h_{ex}, \theta_{RW,hs}, \theta_{HPU,out}, q_{HPU,min})
$$

$q_{HPU,ave}$：1 時間平均のヒートポンプユニットの暖房出力(W)  
$q_{HPU,min}$：1 時間平均のヒートポンプユニットの最小暖房出力(W)  
$P_{HPU,ctn}$：1 時間平均の連続運転時のヒートポンプユニットの消費電力(W)  
$P_{HPU,min}$：1 時間平均のヒートポンプユニットの最小消費電力(W)  
$\theta_{ex}$：1 時間平均の外気温度(℃)  
$h_{ex}$：1 時間平均の外気相対湿度(％)  
$\theta_{RW,hs}$：1 時間平均の熱源機の戻り温水温度(℃)  
$\theta_{HPU,out}$：1 時間平均のヒートポンプユニットの出湯温度(℃)  
$CD$：CD 値  
$r_{intmit}$：ヒートポンプユニットの 1 時間平均の断続運転率

CD 値は0.23とする。

In [ ]:
# 1時間当たりのヒートポンプユニットの消費電力量 (4)
def get_E_E_HPU(q_HPU_ave, q_HPU_min, Theta_ex, h_ex, Theta_RW_hs, Theta_HPU_out):
    """
    :param q_HPU_ave: 1時間平均のヒートポンプユニットの暖房出力 (W)
    :param q_HPU_min: 1時間平均のヒートポンプユニットの最小暖房出力 (W)
    :param Theta_ex: 1時間平均の外気温度 (℃)
    :param h_ex: 1時間平均の外気相対湿度 (%)
    :param Theta_RW_hs: 1時間平均の熱源機の往き温水温度 (℃)
    :param Theta_HPU_out: 1時間平均のヒートポンプユニットの出湯温度 (℃)
    :return: 1時間当たりのヒートポンプユニットの消費電力量 (kWh/h)
    """
    # E_E_HPU の作業領域を確保
    E_E_HPU = np.zeros(24 * 365)

    # ==== 連続運転の場合 ====
    f_ctn = q_HPU_ave >= q_HPU_min

    # 1時間平均の連続運転時のヒートポンプユニットの消費電力 (W) (4a-2)
    P_HPU_ctn = f_HPU_p(Theta_ex, h_ex, Theta_RW_hs, Theta_HPU_out, q_HPU_ave)

    # 1時間当たりのヒートポンプユニットの消費電力量 (kW) (4a-1)
    E_E_HPU[f_ctn] = P_HPU_ctn[f_ctn] * 10 ** (-3)

    # ==== 断続運転の場合 ====
    f_intmit = q_HPU_ave < q_HPU_min

    # 1時間平均のヒートポンプユニットの断続運転率 (5)
    r_intmit = get_r_intmit(q_HPU_ave, q_HPU_min)

    # 1時間平均のヒートポンプユニットの最小消費電力 (kW) (4b-2)
    P_HPU_min = f_HPU_p(Theta_ex, h_ex, Theta_RW_hs, Theta_HPU_out, q_HPU_min)

    # 1時間当たりのヒートポンプユニットの消費電力量 (kW) (4b-1)
    CD = 0.23
    E_E_HPU[f_intmit] = q_HPU_ave[f_intmit] / (
            q_HPU_min[f_intmit] / P_HPU_min[f_intmit] * (1 - CD * (1 - r_intmit[f_intmit]))) * 10 ** (-3)

    return E_E_HPU

1 時間平均のヒートポンプユニットの断続運転率$r_{intmit}$は、1.0を超えない。

$$\large
r_{intmit} = \frac{q_{HPU,ave}}{q_{HPU,min}}
$$

$q_{HPU,ave}$：1 時間平均のヒートポンプユニットの暖房出力(W)  
$q_{HPU,min}$：1 時間平均のヒートポンプユニットの最小暖房出力(W)  

In [ ]:
# 1時間平均のヒートポンプユニットの断続運転率 (5)
def get_r_intmit(q_HPU_ave, q_HPU_min):
    """
    :param q_HPU_ave: 1時間平均のヒートポンプユニットの平均暖房出力 (W)
    :param q_HPU_min: 1時間平均のヒートポンプユニットの最小暖房出力 (W)
    :return: 1時間平均のヒートポンプユニットの断続運転率 (-)
    """
    r_intmit = np.clip(q_HPU_ave / q_HPU_min, None, 1.0)
    return r_intmit

$$\large
q_{HPU,min} = f_{HPU,q,min} (\theta_{ex}, \theta_{RW,hs}, \theta_{HPU,out})
$$

$\theta_{ex}$：1 時間平均の外気温度(℃)  
$\theta_{RW,hs}$：1 時間平均の熱源機の戻り温水温度(℃)  
$\theta_{HPU,out}$：1 時間平均のヒートポンプユニットの出湯温度(℃)  

In [ ]:
# 1時間平均のヒートポンプユニットの最小暖房出力 (6)
def get_q_HPU_min(Theta_ex, Theta_RW_hs, Theta_HPU_out):
    """
    :param Theta_ex: 1時間平均の外気温度 (℃)
    :param Theta_RW_hs: 1時間平均の熱源機の戻り温水温度 (℃)
    :param Theta_HPU_out: 1時間平均のヒートポンプユニットの出湯温度 (℃)
    :return: 1時間平均のヒートポンプユニットの最小暖房出力 (W)
    """
    return f_HPU_q_min(Theta_ex, Theta_RW_hs, Theta_HPU_out)

$$\large
q_{HPU,ave} = Q_{HPU} \times 10^{6} \div 3600
$$

$Q_{HPU}$：1 時間当たりのヒートポンプユニットの暖房出力(MJ/h)  

In [ ]:
# 1時間平均のヒートポンプユニットの平均暖房出力 (7)
def get_q_HPU_ave(Q_HPU):
    """
    :param Q_HPU: 1時間当たりのヒートポンプユニットの暖房出力 (MJ/h)
    :return: 1時間平均のヒートポンプユニットの平均暖房出力 (W)
    """
    return Q_HPU * 10 ** 6 / 3600

#### E.3.3 ヒートポンプユニットの暖房出力

$$\large
Q_{HPU} = Q_{out,hs,HPU} + Q_{GU,loss,HPU} + Q_{TU,tank,loss} + Q_{TU,pipe,loss}
$$

$Q_{out,hs,HPU}$：1 時間当たりの熱源機のヒートポンプユニット分担暖房出力(MJ/h)  
$Q_{GU,loss,HPU}$：1 時間当たりのヒートポンプユニット運転時のガスユニットの熱損失量(MJ/h)  
$Q_{TU,tank,loss}$：1 時間当たりのタンクユニットのタンクの熱損失量(MJ/h)  
$Q_{TU,pipe,loss}$：1 時間当たりのタンクユニットのヒートポンプ配管の熱損失量(MJ/h)  

In [ ]:
# 1時間当たりのヒートポンプユニットの暖房出力 (8)
def get_Q_HPU(Q_out_hs_HPU, Q_GU_loss_HPU, Q_TU_tank_loss, Q_TU_pipe_loss):
    """
    :param Q_out_hs_HPU: 1時間当たりの熱源機のヒートポンプユニット分担暖房出力 (MJ/h)
    :param Q_GU_loss_HPU: 1時間当たりのヒートポンプユニット運転時のガスユニットの熱損失量 (MJ/h)
    :param Q_TU_tank_loss: 1時間当たりのタンクユニットのタンクの熱損失量 (MJ/h)
    :param Q_TU_pipe_loss: 1時間当たりのタンクユニットのヒートポンプ配管の熱損失量 (MJ/h)
    :return: 1時間当たりのヒートポンプユニットの暖房出力 (MJ/h)
    """
    Q_HPU = Q_out_hs_HPU + Q_GU_loss_HPU + Q_TU_tank_loss + Q_TU_pipe_loss
    return Q_HPU

$$\large
Q_{TU,pipe,loss} = q_{TU,pipe,loss} \times 3600 \times 10^{-6}
$$

$$\large
q_{TU,pipe,loss} = f_{TU,q,HPP} (\theta_{SW,hs}, \theta_{ex})
$$

$q_{TU,pipe,loss}$：1 時間平均のタンクユニットのヒートポンプ配管の熱損失(W)  
$\theta_{SW,hs}$：1 時間平均の熱源機の往き温水温度(℃)  
$\theta_{ex}$：1 時間平均の外気温度(℃)  

In [ ]:
# 1時間当たりのタンクユニットのヒートポンプ管の熱損失量 (9)
def get_Q_TU_pipe_loss(Theta_SW_hs, Theta_ex):
    """
    :param Theta_SW_hs: 1時間平均の熱源機の往き温水温度 (℃)
    :param Theta_ex: 外気温度 (℃)
    :return: 1時間当たりのタンクユニットのヒートポンプ管の熱損失量 (MJ/h)
    """
    # タンクユニットのヒートポンプ配管の熱損失 (W) (9b)
    q_TU_pipe_loss = f_TU_q_HPP(Theta_SW_hs, Theta_ex)

    # タンクユニットのヒートポンプ管の熱損失量 (MJ/h) (9a)
    Q_TU_pipe_loss = q_TU_pipe_loss * 3600 * 10 ** (-6)

    return Q_TU_pipe_loss

$$\large
Q_{TU,tank,loss} = q_{TU,tank,loss} \times 3600 \times 10^{-6}
$$

$$\large
q_{TU,tank,loss} = f_{TU,q,tank} (\theta_{HPU,out}, \theta_{TU,amb})
$$

$q_{TU,tank,loss}$：1 時間平均のタンクユニットのタンクの熱損失(W)  
$\theta_{HPU,out}$：1 時間平均のヒートポンプユニットの出湯温度(℃)  
$\theta_{TU,amb}$：1 時間平均のタンクユニットの周囲空気温度(℃)  

In [ ]:
# 1時間当たりのタンクユニットのタンクの熱損失量 (10)
def get_Q_TU_tank_loss(Theta_HPU_out, Theta_TU_amb):
    """
    :param Theta_HPU_out: 1時間平均のヒートポンプユニットの出湯温度 (℃)
    :param Theta_TU_amb: 1時間平均のタンクユニットの周囲空気温度 (℃)
    :return: 1時間当たりのタンクユニットのタンクの熱損失量 (MJ/h)
    """
    # タンクユニットのタンクの熱損失 (W) (10b)
    q_TU_tank_loss = f_TU_q_tank(Theta_HPU_out, Theta_TU_amb)

    # タンクユニットのタンクの熱損失量 (MJ/h) (10a)
    Q_TU_tank_loss = q_TU_tank_loss * 3600 * 10 ** (-6)

    return Q_TU_tank_loss

1 時間平均のヒートポンプユニットの出湯温度$\theta_{HPU,out}$は、1 時間平均の熱源機のヒートポンプユニットの暖
房出力分担率$r_{HPU}$が1.0を下回る場合は85℃とし、1.0に等しいかつ 1 時間平均の熱源機の往き温水温度
$\theta_{SW,hs}$が40℃の場合は65℃とし、1.0に等しいかつ 1 時間平均の熱源機の往き温水温度$\theta_{SW,hs}$が60℃の
場合は85℃とする。 

In [ ]:
# 1時間平均のヒートポンプユニットの出湯温度
def get_Theta_HPU_out(r_HPU, Theta_SW_hs):
    """
    :param r_HPU: 1時間平均のヒートポンプユニット暖房出力分担率 (-)
    :param Theta_SW_hs: 熱源機の往き温水温度 (℃)
    :return: 1時間平均のヒートポンプユニットの出湯温度 (℃)
    """
    Theta_HPU_out = np.zeros(24 * 365)
    Theta_HPU_out[r_HPU < 1.0] = 85.0
    Theta_HPU_out[np.logical_and(r_HPU == 1.0, Theta_SW_hs == 40.0)] = 65.0
    Theta_HPU_out[np.logical_and(r_HPU == 1.0, Theta_SW_hs == 60.0)] = 85.0
    return Theta_HPU_out

$$\large
Q_{GU,loss,HPU} = q_{GU,loss,HPU} \times r_{HPU} \times 3600 \times 10^{-6}
$$

$$\large
q_{GU,loss,HPU} = f_{GU,q,lossHP} (\theta_{SW,hs}, \theta_{TU,amb})
$$

$q_{GU,loss,HPU}$：1 時間平均のヒートポンプユニット運転時のガスユニットの熱損失(W)  
$\theta_{SW,hs}$：1 時間平均の熱源機の往き温水温度(℃)  
$\theta_{TU,amb}$：1 時間平均のタンクユニットの周囲空気温度(℃)  
$r_{HPU}$：1 時間平均のヒートポンプユニットの暖房出力分担率  

In [ ]:
# 1時間当たりのヒートポンプユニット運転時のガスユニットの熱損失量 (11)
def get_Q_GU_loss_HPU(Theta_SW_hs, Theta_TU_amb, r_HPU):
    """
    :param Theta_SW_hs: 熱源機の往き温水温度 (℃)
    :param Theta_TU_amb: 1時間平均のタンクユニットの周囲空気温度 (℃)
    :param r_HPU: 1時間平均のヒートポンプユニット暖房出力分担率 (-)
    :return: 1時間当たりのヒートポンプユニット運転時のガスユニットの熱損失量 (MJ/h)
    """
    # ヒートポンプユニット運転時のガスユニットの熱損失 (W) (11b)
    q_GU_loss_HPU = f_GU_q_lossHP(Theta_SW_hs, Theta_TU_amb)

    # ヒートポンプユニット運転時のガスユニットの熱損失量 (MJ/h) (11a)
    Q_GU_loss_HPU = q_GU_loss_HPU * r_HPU * 3600 * 10 ** (-6)

    return Q_GU_loss_HPU

$$\large
Q_{out,hs,HPU} = Q_{out,hs} \times r_{HPU}
$$

$Q_{out,hs}$：1 時間平均の熱源機の暖房出力(MJ/h)  
$r_{HPU}$：1 時間平均のヒートポンプユニットの暖房出力分担率  

In [ ]:
# 1時間当たりの熱源機のヒートポンプユニット分断暖房出力 (12)
def get_Q_out_hs_HPU(Q_out_hs, r_HPU):
    """
    :param Q_out_hs: 1時間平均の熱源機の暖房出力 (MJ/h)
    :param r_HPU: 1時間平均のヒートポンプユニット暖房出力分担率 (-)
    :return: 1時間当たりの熱源機のヒートポンプユニット分断暖房出力 (MJ/h)
    """
    return Q_out_hs * r_HPU

#### E.3.4 ガスユニットの補機の消費電力

$$\large
E_{E,GU,aux} = (P_{GU,aux,ON} \times r_{GU} + P_{GU,aux,OFF} \times r_{HPU}) \times 10^{-3}
$$

$$\large
P_{GU,aux,ON} = f_{GU,P,aux} (q_{G,GU})
$$

$$\large
P_{GU,aux,OFF} = f_{GU,P,aux} (0.0)
$$

$P_{GU,aux,ON}$：1 時間平均のガス燃焼時のガスユニットの補機の消費電力(W)  
$P_{GU,aux,OFF}$：1 時間平均のガス非燃焼時のガスユニットの補機の消費電力(W)  
$r_{GU}$：1 時間平均のガスユニットの暖房出力分担率  
$r_{HPU}$：1 時間平均のヒートポンプユニットの暖房出力分担率  
$q_{G,GU}$：1 時間平均のガスユニットのガス消費量(W)  

In [ ]:
# 1時間当たりのガスユニットの補機の消費電力 (13)
def get_E_E_GU_aux(q_G_GU, r_HPU, r_GU):
    """
    :param q_G_GU: 1時間平均のガスユニットのガス消費量 (W)
    :param r_HPU: 1時間平均のヒートポンプユニットの暖房出力分担率 (-)
    :return: 1時間当たりのガスユニットの補機の消費電力 (kW)
    """
    # 1時間平均のガス非燃焼時のガスユニットの補機の消費電力 (W) (13c)
    P_GU_aux_OFF = f_GU_P_aux(0.0)

    # 1時間平均のガス燃焼時のガスユニットの補機の消費電力 (W) (13b)
    P_GU_aux_ON = f_GU_P_aux(q_G_GU)

    # 1時間当たりのガスユニットの補機の消費電力 (13a)
    E_E_GU_aux = (P_GU_aux_ON * r_GU + P_GU_aux_OFF * r_HPU) * 10 ** (-3)

    return E_E_GU_aux

#### E.3.5 ガスユニットのガス消費量

$$\large
E_{E,GU} = q_{G,GU} \times r_{GU} \times 3600 \times 10^{-6}
$$

$$\large
q_{G,GU} = f_{GU,G} (q_{GU}, e_{GU,rtd}, q_{GU,rtd})
$$

$$\large
q_{GU} = min(q_{out,hs}, q_{GU,rtd})
$$

$q_{G,GU}$：1 時間平均のガスユニットのガス消費量(W)  
$r_{GU}$：1 時間平均のガスユニットの暖房出力分担率  
$q_{GU}$：1 時間平均のガスユニットの暖房出力(W)  
$q_{out,hs}$：1 時間平均の熱源機の暖房出力(W)  
$e_{GU,rtd}$：ガスユニットの定格暖房効率  
$q_{GU,rtd}$：ガスユニットの定格暖房能力(W)  

In [ ]:
# 1時間平均のガスユニットのガス消費量 (MJ/h) (14a)
def get_E_G_GU(r_GU, q_out_hs, Theta_SW_hs):
    """
    :param r_GU: 1時間平均のガスユニットの暖房出力分担率 (-)
    :param q_out_hs: 1時間平均の熱源機の暖房出力 (W)
    :param Theta_SW_hs: 熱源機の往き温水温度 (℃)
    :return: 1時間平均のガスユニットのガス消費量 (MJ/h)
    """
    # 1時間平均のガスユニットのガス消費量 (14b)
    q_G_GU = get_q_G_GU(q_out_hs, Theta_SW_hs)

    # ガスユニットのガス消費量 (14a)
    E_G_GU = q_G_GU * r_GU * 3600 * 10 ** (-6)

    return E_G_GU


# 1時間平均のガスユニットのガス消費量 (W) (14b)
def get_q_G_GU(q_out_hs, Theta_SW_hs):
    """
    :param q_out_hs: 1時間平均の熱源機の暖房出力 (W)
    :param Theta_SW_hs: 熱源機の往き温水温度 (℃)
    :return: 1時間平均のガスユニットのガス消費量 (W)
    """
    # ガスユニットの定格暖房効率
    e_GU_rtd = get_e_GU_rtd()

    # ガスユニットの定格暖房能力
    q_GU_rtd = get_q_GU_rtd()

    # ガスユニットの暖房能力
    q_GU = get_q_GU(q_out_hs, q_GU_rtd)

    return f_GU_G(q_GU, e_GU_rtd, q_GU_rtd, Theta_SW_hs)


# ガスユニットの暖房能力 (14c)
def get_q_GU(q_out_hs, q_GU_rtd):
    """
    :param q_out_hs: 1時間平均の熱源機の暖房出力 (W)
    :param q_GU_rtd: ガスユニットの定格暖房効率 (-)
    :return: ガスユニットの暖房能力 (W)
    """
    return np.clip(q_out_hs, None, q_GU_rtd)

ガスユニットの定格暖房効率$e_{rtd}$は0.87に等しいとし、ガスユニットの定格暖房能力$q_{GU,rtd}$は14000に等しいとする。 

In [ ]:
# ガスユニットの定格暖房効率 (-)
def get_e_GU_rtd():
    return 0.87


# ガスユニットの定格暖房能力 (W)
def get_q_GU_rtd():
    return 14000

#### E.3.6 暖房出力分担率

1 時間平均のヒートポンプユニットの暖房出力分担率$r_{HPU}$は0を下回る場合は0とし、1を上回る場合は1とする。 

$$\large
r_{GU} = 1 - r_{HPU}
$$

$$\large
r_{HPU} = \frac{q_{HPU,max} - q'_{TU,tank,loss} - q'_{TU,pipe,loss}}{q_{out,hs} + q'_{GU,loss,HPU}}
$$

$q_{HPU,max}$：1 時間平均のヒートポンプユニットの最大暖房出力(W)  
$q'_{TU,tank,loss}$：1 時間平均のヒートポンプユニット最大運転時のタンクユニットのタンクの熱損失(W)  
$q'_{TU,pipe,loss}$：1 時間平均のヒートポンプユニット最大運転時のタンクユニットのヒートポンプ配管の熱損失(W)  
$q_{out,hs}$：1 時間平均の熱源機の暖房出力(W)  
$q'_{GU,loss,HPU}$：1 時間平均のヒートポンプユニット最大運転時のヒートポンプユニット運転時のガスユニットの熱損失(W)  

In [ ]:
# 1時間平均のガスユニットの暖房出力分担率 (15)
def get_r_GU(r_HPU):
    """
    :param r_HPU: 1時間平均のヒートポンプユニットの暖房出力分担率 (-)
    :return: 1時間平均のガスユニットの暖房出力分担率 (-)
    """
    return 1 - r_HPU

# 1時間平均のヒートポンプユニットの暖房出力分担率 (16)
def get_r_HPU(q_HPU_max, q_dash_TU_tank_loss, q_dash_TU_pipe_loss, q_out_hs, q_dash_GU_loss_HPU):
    """
    :param q_HPU_max: 1時間平均のヒートポンプユニットの最大暖房出力 (W)
    :param q_dash_TU_tank_loss: 1時間平均のヒートポンプユニット最大運転時のタンクユニットのタンクの熱損失 (W)
    :param q_dash_TU_pipe_loss: 1時間平均のヒートポンプユニット最大運転時のタンクユニットのヒートポンプ配置の熱損失 (W)
    :param q_out_hs: 1時間平均の熱源機の暖房出力 (W)
    :param q_dash_GU_loss_HPU: 1時間平均のヒートポンプユニット最大運転時のヒートポンプユニット運転時のガスユニットの熱損失 (W)
    :return: 1時間平均のヒートポンプユニットの暖房出力分担率 (-)
    """
    r_HPU = np.zeros(24*365)
    f = (q_out_hs + q_dash_GU_loss_HPU > 0)
    r_HPU[f] = (q_HPU_max[f] - q_dash_TU_tank_loss[f] - q_dash_TU_pipe_loss[f]) / (q_out_hs[f] + q_dash_GU_loss_HPU[f])
    return np.clip(r_HPU, 0, 1)

$$\large
q_{HPU,max} = f_{HPU,q,max} (\theta_{ex}, \theta_{RW,hs}, \theta'_{HPU,out})
$$

$\theta_{ex}$：1 時間平均の外気温度(℃)  
$\theta_{RW,hs}$：1 時間平均の熱源機の戻り温水温度(℃)  
$\theta'_{HPU,out}$：1 時間平均のヒートポンプ最大運転時のヒートポンプユニットの出湯温度(℃)

In [ ]:
# 1時間平均のヒートポンプユニットの最大暖房出力 (17)
def get_q_HPU_max(Theta_ex, Theta_RW_hs):
    """
    :param Theta_ex: 1時間平均の外気温度 (℃)
    :param Theta_RW_hs: 1時間平均の熱源機の戻り温水温度 (℃)
    :return: ヒートポンプユニットの最大暖房出力 (W)
    """
    # 1時間平均のヒートポンプ大運転時のヒートポンプユニットの出湯温度
    Theta_dash_HPU_out = get_Theta_dash_HPU_out()

    # ヒートポンプユニットの最大暖房出力を求める関数 (26)
    q_HPU_max = f_HPU_q_max(Theta_ex, Theta_RW_hs, Theta_dash_HPU_out)

    return q_HPU_max

$$\large
q'_{TU,pipe,loss} = f_{TU,q,HPP} (\theta_{SW,hs}, \theta_{ex})
$$

$\theta_{SW,hs}$：1 時間平均の熱源機の往き温水温度(℃)  
$\theta_{ex}$：1 時間平均の外気温度(℃)  

In [ ]:


# 1時間平均のヒートポンプ最大運転時のタンクユニットのヒートポンプ配管の熱損失 (18)
def get_q_dash_TU_pipe_loss(Theta_SW_hs, Theta_ex):
    """
    :param Theta_SW_hs: 1時間平均の熱源機の往き温水温度 (℃)
    :param Theta_ex: 1時間平均の外気温度 (℃)
    :return: 1時間平均のヒートポンプ最大運転時のタンクユニットのヒートポンプ配管の熱損失 (W)
    """
    # タンクユニットのヒートポンプ配管の熱損失を求める関数 (28)
    q_dash_TU_pipe_loss = f_TU_q_HPP(Theta_SW_hs, Theta_ex)

    return q_dash_TU_pipe_loss

$$\large
q'_{TU,tank,loss} = f_{TU,q,tank} (\theta'_{HPU,out}, \theta_{TU,amb})
$$

$\theta'_{HPU,out}$：1 時間平均のヒートポンプ最大運転時のヒートポンプユニットの出湯温度(℃)  
$\theta_{TU,amb}$：1 時間平均のタンクユニットの周囲空気温度(℃)  

In [ ]:
# ヒートポンプ最大運転時のタンクユニットのタンクの熱損失 (19)
def get_q_dash_TU_tank_loss(Theta_TU_amb):
    """
    :param Theta_TU_amb: 1時間平均のタンクユニットの周囲空気温度 (℃)
    :return: ヒートポンプ最大運転時のタンクユニットのタンクの熱損失 (W)
    """
    # 1時間平均のヒートポンプ最大運転時のヒートポンプユニットの出湯温度
    Theta_dash_HPU_out = get_Theta_dash_HPU_out()

    # タンクユニットのタンクの熱損失を求める関数 (29)
    q_dash_TU_tank_loss = f_TU_q_tank(Theta_dash_HPU_out, Theta_TU_amb)

    return q_dash_TU_tank_loss

$$\large
q'_{GU,loss,HPU} = f_{GU,q,lossHP} (\theta_{SW,hs}, \theta_{TU,amb})
$$

$\theta_{SW,hs}$：1 時間平均の熱源機の往き温水温度(℃)  
$\theta_{TU,amb}$：1 時間平均のタンクユニットの周囲空気温度(℃)  

In [ ]:
# ヒートポンプ最大運転時のヒートポンプユニット運転時のガスユニットの熱損失 (20)
def get_q_dash_GU_loss_HPU(Theta_SW_hs, Theta_TU_amb):
    """
    :param Theta_SW_hs: 1時間平均の熱源機の往き温水温度 (℃)
    :param Theta_TU_amb: 1時間平均のタンクユニットの周囲空気温度 (℃)
    :return: ヒートポンプ最大運転時のヒートポンプユニット運転時のガスユニットの熱損失 (W)
    """
    # ヒートポンプユニット運転時のガスユニットの熱損失を求める関数 (32)
    q_dash_GU_loss_HPU = f_GU_q_lossHP(Theta_SW_hs, Theta_TU_amb)

    return q_dash_GU_loss_HPU

1 時間平均のヒートポンプ最大運転時のヒートポンプユニットの出湯温度$\theta'_{HPU,out}$ は 85℃する。 

In [ ]:
# 1時間平均のヒートポンプ最大運転時のヒートポンプユニットの出湯温度 (℃)
def get_Theta_dash_HPU_out():
    return 85.0

#### E.3.7 熱源機の平均暖房出力及び熱源機最大暖房出力

$$\large
q_{out,hs} = Q_{out,hs} \times 10^{6} \div 3600
$$

$Q_{out,hs}$：1時間当たりの熱源機の暖房出力(MJ/h)  

In [ ]:
# 1時間平均の熱源機の暖房出力 (21)
def get_q_out_hs(Q_out_hs):
    """
    :param Q_out_hs: 1時間当たりの熱源機の暖房出力 (MJ/h)
    :return: 1時間平均の熱源機の暖房出力 (W)
    """
    return Q_out_hs * 10 ** 6 / 3600

1 時間当たりの熱源機の暖房出力$Q_{out,hs}$は、本文により定義される、1 時間当たりの熱源機の最大暖房出力$Q_{max,hs}$を超えない範囲で 1 時間当たりの温水熱需要$Q_{dmd,hs}$に等しい。 

$$\large
Q_{max,hs} = q_{GU,rtd} \times 3600 \times 10^{-6}
$$

$q_{GU,rtd}$：ガスユニットの定格暖房能力(W)  

In [ ]:
# 1時間当たりの熱源機の最大暖房出力 (22)
def get_Q_max_hs(q_GU_rtd):
    """
    :param q_GU_rtd: ガスユニットの定格暖房能力 (W)
    :return: 1時間当たりの熱源機の最大暖房出力 (MJ/h)
    """
    Q_max_hs = q_GU_rtd * 3600 * 10 ** (-6)
    return Q_max_hs

#### E.3.8 タンクユニット周囲の空気温度

タンクユニットを室内に設置する場合

$$\large
\theta_{TU,amb} = 0.25 \times \theta_{ex} + 0.75 \times \theta_{in} 
$$

タンクユニットを室外に設置する場合

$$\large
\theta_{TU,amb} = \theta_{ex}
$$

$\theta_{ex}$：1 時間平均の外気温度(℃)  
$\theta_{in}$：1 時間平均の室内温度(℃)  

In [ ]:
# 1時間平均のタンクユニットの周囲空気温度 (23)
def get_Theta_TU_amb(Theta_ex, TU_place):
    """
    :param Theta_ex: 1時間平均の外気温度 (℃)
    :param TU_place: タンクユニットの設置場所
    :return: 1時間平均のタンクユニットの周囲空気温度 (℃)
    """
    if TU_place == '屋内':
        # 1時間平均の室内温度
        Theta_in = get_Theta_in()
        return 0.25 * Theta_ex + 0.75 * Theta_in  # (23a)
    elif TU_place == '屋外':
        return Theta_ex  # (23b)
    else:
        raise ValueError(TU_place)

暖房運転時において、1 時間平均の室内温度$\theta_{in}$は、20℃とする。

In [ ]:
# 暖房時運転における1時間平均の室内温度 (℃)
def get_Theta_in():
    return 20.0

#### E.4 評価関数

#### E.4.1 ヒートポンプユニット 

$$
f_{HPU,P} (\theta_{ex}, h_{ex},\theta_{RW,hs},\theta_{HP,out}, q_{HPU,out}) = (0.1869 \times P^{2} + 0.2963 \times P + 0.565) \times 10^{3}
$$

$$
P =  0.42108 \times q_{HPU,out} \times 10^{-3} - 0.03889 \times \theta_{ex} - 0.00762 \times h_{ex} +  0.03313 \times \theta_{RW,hs}\\ + 0.00449 \times \theta_{HP,out} - 0.82066
$$

$\theta_{ex}$：外気温度(℃)  
$h_{ex}$：外気相対湿度(％)  
$q_{HPU,out}$：ヒートポンプユニットの平均暖房出力(W)  
$\theta_{RW,hs}$：熱源機の戻り温水温度(℃)  
$\theta_{HP,out}$：ヒートポンプユニットの出湯温度(℃)  

In [ ]:
# ヒートポンプユニットの消費電力を求める関数 (24)
def f_HPU_p(Theta_ex, h_ex, Theta_RW_hs, Theta_HP_out, q_HPU_out):
    """
    :param Theta_ex: 外気温度 (℃)
    :param h_ex: 外気相対湿度 (%)
    :param Theta_RW_hs: 熱源機の戻り温水温度 (℃)
    :param Theta_HP_out: ヒートポンプユニットの出湯温度 (℃)
    :param q_HPU_out: ヒートポンプユニットの平均暖房出力 (W)
    :return: ヒートポンプユニットの消費電力 (W)
    """
    P = 0.42108 * q_HPU_out * 10 ** (-3) \
        - 0.03889 * Theta_ex \
        - 0.00762 * h_ex \
        + 0.03313 * Theta_RW_hs \
        + 0.00449 * Theta_HP_out \
        - 0.82066
    return (0.1869 * P ** 2 + 0.2963 * P + 0.565) * 10 ** 3

ヒートポンプユニットの最小暖房出力

$$
f_{HPU,q,min} (\theta_{ex}, \theta_{RW,hs},\theta_{HPU,out}) = 6.08 \times \theta_{ex} - 46.56 \times \theta_{RW,hs} +0.04 \times \theta_{HPU,out} + 4739.62
$$

In [ ]:
# ヒートポンプユニットの最小暖房出力を求める関数 (25)
def f_HPU_q_min(Theta_ex, Theta_RW_hs, Theta_HPU_out):
    """
    :param Theta_ex: 外気温度 (℃)
    :param Theta_RW_hs: 熱源機の戻り温水温度 (℃)
    :param Theta_HPU_out: ヒートポンプユニットの出湯温度 (℃)
    :return: ヒートポンプユニットの最小暖房出力 (W)
    """
    return 6.08 * Theta_ex - 46.56 * Theta_RW_hs + 0.04 * Theta_HPU_out + 4739.62

ヒートポンプユニットの最大暖房出力

$$
f_{HPU,q,max} (\theta_{ex}, \theta_{RW,hs},\theta_{HPU,out}) = 1.36 \times \theta_{ex} - 89.85 \times \theta_{RW,hs} +  46.5 \times \theta_{HPU,out} +  5627.07
$$

$\theta_{ex}$：外気温度(℃)  
$\theta_{RW,hs}$：熱源機の戻り温水温度(℃)  
$\theta_{HPU,out}$：ヒートポンプユニットの出湯温度(℃)  

In [ ]:
# ヒートポンプユニットの最大暖房出力を求める関数 (26)
def f_HPU_q_max(Theta_ex, Theta_RW_hs, Theta_HPU_out):
    return 1.36 * Theta_ex - 89.85 * Theta_RW_hs + 46.5 * Theta_HPU_out + 5627.07

#### E.4.2 タンクユニット 

$$\large
f_{TU,P,aux} (r_{intmit}) = 5 + 10 + r_{intmit}
$$

$r_{intmit}$：ヒートポンプユニットの断続運転率  

In [ ]:
# タンクユニットの補機の消費電力を求める関数 (27)
def f_TU_P_aux(r_intmit):
    """
    :param r_intmit: ヒートポンプユニットの断続運転率
    :return: タンクユニットの補機の消費電力 (W)
    """
    return 5 + 10 * r_intmit

$$\large
f_{TU,q,HPP} (\theta_{SW,hs}, \theta_{ex}) = 2.8 \times (\theta_{SW,hs} - \theta_{ex})
$$

$\theta_{SW,hs}$：熱源機の往き温水温度(℃)  
$\theta_{ex}$：外気温度(℃)  

In [ ]:
# タンクユニットのヒートポンプ配管の熱損失を求める関数 (28)
def f_TU_q_HPP(Theta_SW_hs, Theta_ex):
    """
    :param Theta_SW_hs: 熱源機の往き温水温度 (℃)
    :param Theta_ex: 外気温度 (℃)
    :return: タンクユニットのヒートポンプ配管の熱損失 (W)
    """
    return 2.8 * (Theta_SW_hs - Theta_ex)

$$\large
f_{TU,q,tank} (\theta_{HP,out}, \theta_{TU,amb}) = max(0, 1.6 \times (\theta_{HP,out} - \theta_{TU,amb})-72.1)
$$

$\theta_{HP,out}$：ヒートポンプユニットの出湯温度(℃)  
$\theta_{TU,amb}$：タンクユニットの周囲空気温度(℃)  

In [ ]:
# タンクユニットのタンクの熱損失を求める関数 (29)
def f_TU_q_tank(Theta_HP_out, Theta_TU_amb):
    """
    :param Theta_HP_out: ヒートポンプユニットの出湯温度 (℃)
    :param Theta_TU_amb: タンクユニットの周囲空気温度 (℃)
    :return: タンクユニットのタンクの熱損失 (W)
    """
    return np.clip(1.6 * (Theta_HP_out - Theta_TU_amb) - 72.1, 0, None)

#### E.4.3 ガスユニット 

$$\large
f_{GU,G} (q_{GU},e_{GU,rtd},q_{GU,rtd}) = (q_{GU} + q_{body}) \div e_{ex}
$$

$$\large
e_{ex} = e_{GU,rtd} \times f_{ex} \times (q_{GU,rtd} + q_{body}) \div q_{GU,rtd}
$$

$q_{GU}$：ガスユニットの暖房出力(W)  
$q_{body}$：ガスユニットの筐体放熱損失(W)  
$e_{ex}$：ガスユニット燃焼室の熱交換効率  
$f_{ex}$：ガスユニットの定格暖房効率を補正する係数  
$e_{GU,rtd}$：ガスユニットの定格暖房効率  
$q_{GU,rtd}$：ガスユニットの定格暖房能力(W)  

In [ ]:
# ガスユニットのガス消費量を求める関数 (30,31)
def f_GU_G(q_GU, e_GU_rtd, q_GU_rtd, Theta_SW_hs):
    """
    :param q_GU: ガスユニットの暖房出力 (W)
    :param q_GU_rtd: ガスユニットの定格暖房能力 (W)
    :param e_GU_rtd: ガスユニットの定格暖房効率 (-)
    :param Theta_SW_hs: 熱源機の往き温水温度 (℃)
    :return: ガスユニットのガス消費量
    """
    # f_ex, q_body の作業領域確保
    f_ex = np.zeros(24 * 365)
    q_body = np.zeros(24 * 365)

    # Theta_SW_hs == 40 の場合
    f1 = Theta_SW_hs == 40
    f_ex[f1] = table_e_3[0][0]
    q_body[f1] = table_e_3[1][0]

    # Theta_SW_hs == 60 の場合
    f2 = Theta_SW_hs == 60
    f_ex[f2] = table_e_3[0][1]
    q_body[f2] = table_e_3[1][1]

    e_ex = e_GU_rtd * f_ex * (q_GU_rtd + q_body) / q_GU_rtd

    return (q_GU + q_body) / e_ex

<div style="text-align: center;font-weight: bold;">表　ガスユニットの定格暖房効率を補正する係数及びガスユニットの筐体放熱損失</div>

<table>
<thead>
<tr>
<th rowspan="2"></th>
<th colspan="2">熱源機の往き温水温度 $\theta_{SW,hs}$</th>
</tr>
<tr>
<th>40℃の場合</th>
<th>60℃の場合</th>
</tr>
</thead>

<tbody>
<tr>
<th>ガスユニットの定格暖房効率を補正する係数 $f_{ex}$</th>
<td>1.064</td>
<td>1.038</td>
</tr>
<tr>
<th>ガスユニットの筐体放熱損失 $q_{body}$</th>
<td>123.74</td>
<td>225.26</td>
</tr>

</tbody>
</table>


In [ ]:
# 表E.3 ガスユニットの
table_e_3 = [
    (1.064, 1.038),
    (123.74, 225.26)
]

$$\large
f_{GU,q,lossHP} (\theta_{SW,hs}, \theta_{TU,amb}) = max(5 \times (\theta_{SW,hs} - \theta_{TU,amb}) -100, 0)
$$

$\theta_{SW,hs}$：熱源機の往き温水温度(℃)  
$\theta_{TU,amb}$：タンクユニットの周囲空気温度(℃)  

In [ ]:
# ヒートポンプユニット運転時のガスユニットの熱損失を求める関数 (32)
def f_GU_q_lossHP(Theta_SW_hs, Theta_TU_amb):
    """
    :param Theta_SW_hs: 熱源機の往き温水温度 (℃)
    :param Theta_TU_amb: タンクユニットの周囲空気温度 (℃)
    :return: ヒートポンプユニット運転時のガスユニットの熱損失
    """
    return np.clip(5 * (Theta_SW_hs - Theta_TU_amb) - 100, 0, None)

$$\large
f_{GU,P,aux} (q_{G,GU}) =73 + 0.003 \times q_{G,GU}
$$

$q_{G,GU}$：ガスユニットのガス消費量(W)  

In [ ]:
# ガスユニットの補機の消費電力を求める関数 (33)
def f_GU_P_aux(q_G_GU):
    """
    :param q_G_GU: ガスユニットのガス消費量 (W)
    :return: ガスユニットの補機の消費電力 (W)
    """
    return 73 + 0.003 * q_G_GU

#### E.5 電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機（給湯熱源：ガス瞬間式、暖房熱源：電気ヒートポンプ・ガス瞬間式併用）の定義

省略

#### E.6 品番の追加（参考）

省略